<a href="https://colab.research.google.com/github/chrisoyer/thinkful_notes/blob/master/Unit_4_Spark/Amazon_Kindle_Reviews__Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description:
Challenge: Amazon Reviews Sentiment Analysis

### Initialization

In [0]:
#environment
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_CLASSPATH"] = '/content/spark-2.4.4-bin-hadoop2.7'

In [3]:
# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 119kB/s 
     |████████████████████████████████| 204kB 39.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=0490264dada1b8962b3cccb7d605a9332a826bdd3994cb45d012e4cb3b2240e7
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Spark NLP') \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.2.2") \
    .getOrCreate()

In [0]:
source_url = r"http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz"

I will be using gDrive file location instead of downloading the source file.

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
local_souce_file = r"/content/gdrive/My Drive/thinkful/colab_datasets/reviews_Grocery_and_Gourmet_Food_5.json.gz"
SPARK_URL = "local[*]"
APP_NAME  = "amazon_food_reviews"

In [0]:
from pyspark import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
import pyspark.sql.functions as F

In [0]:
#spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
# not working# !gzip -dkc $local_source_file > /content/reviews
reviews = r'/content/gdrive/My Drive/thinkful/colab_datasets/reviews_Grocery_and_Gourmet_Food_5.json'
reviews_df = spark.read.options(inferschema = "true").json(reviews)

In [0]:
!file $local_source_file

/bin/bash: file: command not found


In [11]:
reviews_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [12]:
reviews_df.show(5)

+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|   reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|616719923X| [0, 0]|    4.0|Just another flav...| 06 1, 2013|A1VEELTKS8NLZB|Amazon Customer|          Good Taste|    1370044800|
|616719923X| [0, 1]|    3.0|I bought this on ...|05 19, 2014|A14R9XMZVJ6INB|        amf0001|3.5 stars,  sadly...|    1400457600|
|616719923X| [3, 4]|    4.0|Really good. Grea...| 10 8, 2013|A27IQHDZFQFNGG|        Caitlin|                Yum!|    1381190400|
|616719923X| [0, 0]|    5.0|I had never had i...|05 20, 2013|A31QY5TASILE89|   DebraDownSth|Unexpected flavor...|    1369008000|
|616719923X| [1, 2]|    4.0|I've been looking...|05 26, 2013|A2LWK003FFMCI5|       Diana X.|Not a

In [13]:
!pip install spark-nlp
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher
sparknlp.start()
pipeline = PretrainedPipeline(name='analyze_sentiment', lang='en')

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]


In [14]:
dir(pipeline)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'annotate',
 'fullAnnotate',
 'light_model',
 'model',
 'transform']